In [2]:
import argparse
import os
import sys
import datetime
import time
import math
import json
from pathlib import Path

import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
# from torchvision import datasets, transforms
#from torchvision import models as torchvision_models
import mambular
import dino.utils as utils
from torch.utils.data import Dataset, DataLoader
# import vision_transformer as vits
from dino.vision_transformer import DINOHead

In [3]:
mamba_archs = sorted(mambular.models.__all__)

In [4]:
def get_args_parser():
    parser = argparse.ArgumentParser('DINO', add_help=False)

    # Model parameters
    parser.add_argument('--arch', default='MLPRegressor', type=str,
        choices=mamba_archs,
        help="""Name of architecture to train. For quick experiments with ViTs,
        we recommend using vit_tiny or vit_small.""")
    parser.add_argument('--patch_size', default=16, type=int, help="""Size in pixels
        of input square patches - default 16 (for 16x16 patches). Using smaller
        values leads to better performance but requires more memory. Applies only
        for ViTs (vit_tiny, vit_small and vit_base). If <16, we recommend disabling
        mixed precision training (--use_fp16 false) to avoid unstabilities.""")
    parser.add_argument('--out_dim', default=65536, type=int, help="""Dimensionality of
        the DINO head output. For complex and large datasets large values (like 65k) work well.""")
    parser.add_argument('--norm_last_layer', default=True, type=utils.bool_flag,
        help="""Whether or not to weight normalize the last layer of the DINO head.
        Not normalizing leads to better performance but can make the training unstable.
        In our experiments, we typically set this paramater to False with vit_small and True with vit_base.""")
    parser.add_argument('--momentum_teacher', default=0.996, type=float, help="""Base EMA
        parameter for teacher update. The value is increased to 1 during training with cosine schedule.
        We recommend setting a higher value with small batches: for example use 0.9995 with batch size of 256.""")
    parser.add_argument('--use_bn_in_head', default=False, type=utils.bool_flag,
        help="Whether to use batch normalizations in projection head (Default: False)")

    # Temperature teacher parameters
    parser.add_argument('--warmup_teacher_temp', default=0.04, type=float,
        help="""Initial value for the teacher temperature: 0.04 works well in most cases.
        Try decreasing it if the training loss does not decrease.""")
    parser.add_argument('--teacher_temp', default=0.04, type=float, help="""Final value (after linear warmup)
        of the teacher temperature. For most experiments, anything above 0.07 is unstable. We recommend
        starting with the default value of 0.04 and increase this slightly if needed.""")
    parser.add_argument('--warmup_teacher_temp_epochs', default=0, type=int,
        help='Number of warmup epochs for the teacher temperature (Default: 30).')

    # Training/Optimization parameters
    parser.add_argument('--use_fp16', type=utils.bool_flag, default=True, help="""Whether or not
        to use half precision for training. Improves training time and memory requirements,
        but can provoke instability and slight decay of performance. We recommend disabling
        mixed precision if the loss is unstable, if reducing the patch size or if training with bigger ViTs.""")
    parser.add_argument('--weight_decay', type=float, default=0.04, help="""Initial value of the
        weight decay. With ViT, a smaller value at the beginning of training works well.""")
    parser.add_argument('--weight_decay_end', type=float, default=0.4, help="""Final value of the
        weight decay. We use a cosine schedule for WD and using a larger decay by
        the end of training improves performance for ViTs.""")
    parser.add_argument('--clip_grad', type=float, default=3.0, help="""Maximal parameter
        gradient norm if using gradient clipping. Clipping with norm .3 ~ 1.0 can
        help optimization for larger ViT architectures. 0 for disabling.""")
    parser.add_argument('--batch_size_per_gpu', default=64, type=int,
        help='Per-GPU batch-size : number of distinct images loaded on one GPU.')
    parser.add_argument('--epochs', default=100, type=int, help='Number of epochs of training.')
    parser.add_argument('--freeze_last_layer', default=1, type=int, help="""Number of epochs
        during which we keep the output layer fixed. Typically doing so during
        the first epoch helps training. Try increasing this value if the loss does not decrease.""")
    parser.add_argument("--lr", default=0.0005, type=float, help="""Learning rate at the end of
        linear warmup (highest LR used during training). The learning rate is linearly scaled
        with the batch size, and specified here for a reference batch size of 256.""")
    parser.add_argument("--warmup_epochs", default=10, type=int,
        help="Number of epochs for the linear learning-rate warm up.")
    parser.add_argument('--min_lr', type=float, default=1e-6, help="""Target LR at the
        end of optimization. We use a cosine LR schedule with linear warmup.""")
    parser.add_argument('--optimizer', default='adamw', type=str,
        choices=['adamw', 'sgd', 'lars'], help="""Type of optimizer. We recommend using adamw with ViTs.""")
    parser.add_argument('--drop_path_rate', type=float, default=0.1, help="stochastic depth rate")

    # Multi-crop parameters
    parser.add_argument('--global_crops_scale', type=float, nargs='+', default=(0.4, 1.),
        help="""Scale range of the cropped image before resizing, relatively to the origin image.
        Used for large global view cropping. When disabling multi-crop (--local_crops_number 0), we
        recommand using a wider range of scale ("--global_crops_scale 0.14 1." for example)""")
    parser.add_argument('--local_crops_number', type=int, default=8, help="""Number of small
        local views to generate. Set this parameter to 0 to disable multi-crop training.
        When disabling multi-crop we recommend to use "--global_crops_scale 0.14 1." """)
    parser.add_argument('--local_crops_scale', type=float, nargs='+', default=(0.05, 0.4),
        help="""Scale range of the cropped image before resizing, relatively to the origin image.
        Used for small local view cropping of multi-crop.""")

    # Misc
    parser.add_argument('--data_path', default='/path/to/imagenet/train/', type=str,
        help='Please specify path to the ImageNet training data.')
    parser.add_argument('--output_dir', default=".", type=str, help='Path to save logs and checkpoints.')
    parser.add_argument('--saveckp_freq', default=20, type=int, help='Save checkpoint every x epochs.')
    parser.add_argument('--seed', default=0, type=int, help='Random seed.')
    parser.add_argument('--num_workers', default=10, type=int, help='Number of data loading workers per GPU.')
    parser.add_argument("--dist_url", default="env://", type=str, help="""url used to set up
        distributed training; see https://pytorch.org/docs/stable/distributed.html""")
    parser.add_argument("--local_rank", default=0, type=int, help="Please ignore and do not set this argument.")
    return parser

In [5]:
class DataAugmentationPV(object):
    def __init__(self, masking_percent_student, masking_percent_teacher, data_size):
        # flip_and_color_jitter = transforms.Compose([
        #     transforms.RandomHorizontalFlip(p=0.5),
        #     transforms.RandomApply(
        #         [transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1)],
        #         p=0.8
        #     ),
        #     transforms.RandomGrayscale(p=0.2),
        # ])
        # normalize = transforms.Compose([
        #     transforms.ToTensor(),
        #     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        # ])

        # # first global crop
        # self.global_transfo1 = transforms.Compose([
        #     transforms.RandomResizedCrop(224, scale=global_crops_scale, interpolation=Image.BICUBIC),
        #     flip_and_color_jitter,
        #     utils.GaussianBlur(1.0),
        #     normalize,
        # ])
        # # second global crop
        # self.global_transfo2 = transforms.Compose([
        #     transforms.RandomResizedCrop(224, scale=global_crops_scale, interpolation=Image.BICUBIC),
        #     flip_and_color_jitter,
        #     utils.GaussianBlur(0.1),
        #     utils.Solarization(0.2),
        #     normalize,
        # ])
        # # transformation for the local small crops
        # self.local_crops_number = local_crops_number
        # self.local_transfo = transforms.Compose([
        #     transforms.RandomResizedCrop(96, scale=local_crops_scale, interpolation=Image.BICUBIC),
        #     flip_and_color_jitter,
        #     utils.GaussianBlur(p=0.5),
        #     normalize,
        # ])
        self.student_masking = torch.rand(data_size) < masking_percent_student
        #self.student_data = data * mask
        self.teacher_masking = torch.rand(data_size) < masking_percent_teacher

    def __call__(self, data):
        crops = []
        crops.append(self.student_masking * data)
        crops.append(self.teacher_masking * data)
        # for _ in range(self.local_crops_number):
        #     crops.append(self.local_transfo(image))
        return crops

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split 
df = pd.read_csv('clean_data.csv')
x = df.iloc[:, :-1]
y = df.iloc[:, -1]
x_train_r, x_test, y_train_r, y_test = train_test_split(x, y, train_size=0.9, shuffle=False)
x_train, x_val, y_train, y_val = train_test_split(x_train_r, y_train_r, train_size=5/6, shuffle=False)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler().fit(x_train)
x_train = sc.transform(x_train)
x_val = sc.transform(x_val)
x_test = sc.transform(x_test)

In [ ]:
class TabularDataset(Dataset):
    def __init__(self, data, target_col, mask_studetn, mask_teacher):
        self.data = data.drop(target_col, axis=1)
        self.mask = DataAugmentationPV()
        self.target = data[target_col]
        self.transforms = transforms

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data.iloc[idx].values.astype('float32')
        y = self.target.iloc[idx]

        if self.transforms:
            x = self.transforms(x)

        return torch.tensor(x), torch.tensor(y)

In [37]:
transform = DataAugmentationPV(
        0.3, 1, df.shape
    )

In [66]:
transform.teacher_masking

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])

In [13]:
## teacher and student networks
def st_te_nets():
    student = mambular.models.MLPRegressor()
    teacher = mambular.models.MLPRegressor()
    embed_dim = 64

    # multi-crop wrapper handles forward with inputs of different resolutions
    student = utils.MultiCropWrapper(student, DINOHead(
        embed_dim,
        65536,
        use_bn=False,
        norm_last_layer=True,
    ))
    teacher = utils.MultiCropWrapper(
        teacher,
        DINOHead(embed_dim, 65536, False),
    )
    # move networks to gpu
    student, teacher = student.cuda(), teacher.cuda()
    return student, teacher

In [14]:
s, t = st_te_nets()

/home/m16r2/anaconda3/envs/farid/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
